In [2]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import unidecode
import codecs

# import spacy
# from wordcloud import WordCloud, STOPWORDS
# from fuzzywuzzy import fuzz
# import cudf
# import cupy
# from cuml.feature_extraction.text import TfidfVectorizer
# from cuml.common.sparsefuncs import csr_row_normalize_l2

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

import gc

from tools import *

In [1]:
np.random.seed(0)
nlp = spacy.load('en')#, disable=["tagger", "parser", "ner"])

NameError: name 'np' is not defined

# Getting started with data

In [3]:
source_path = '../input/shopee-product-matching'

In [4]:
train_df = pd.read_csv(f'{source_path}/train.csv')
test_df = pd.read_csv(f'{source_path}/test.csv')
sample_submission_df = pd.read_csv(f'{source_path}/sample_submission.csv')

In [5]:
print(f'Is there any NaN values?: {train_df.isnull().values.any()}')

Is there any NaN values?: False


In [6]:
train_df['image_path'] = train_df['image'].apply(lambda x: f"{source_path}/train_images/{x}")
test_df['image_path'] = test_df['image'].apply(lambda x: f"{source_path}/test_images/{x}")

In [7]:
train_df['hash'] = train_df['image_phash'].apply(lambda x: bin(int(x, 16))[2:].zfill(64))
test_df['hash'] = train_df['image_phash'].apply(lambda x: bin(int(x, 16))[2:].zfill(64))

In [8]:
print(f'Trainset: {train_df.shape} \nTestset: {test_df.shape}')

Trainset: (34250, 7) 
Testset: (3, 6)


In [9]:
train_df.head()

,posting_id,image,image_phash,title,label_group,image_path,hash
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,../input/shopee-product-matching/train_images/...,1001010010010111010011111001001101111101010011...
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,../input/shopee-product-matching/train_images/...,1010111100111111100101000110000011000010100000...
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,../input/shopee-product-matching/train_images/...,1011100101001100101100000000111011010011111001...
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,../input/shopee-product-matching/train_images/...,1000010100010100111111000101100011101010111111...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,../input/shopee-product-matching/train_images/...,1010011011110011000110011111100100100100101011...


In [10]:
test_df.head()

,posting_id,image,image_phash,title,image_path,hash
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...,../input/shopee-product-matching/test_images/0...,1001010010010111010011111001001101111101010011...
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...,../input/shopee-product-matching/test_images/0...,1010111100111111100101000110000011000010100000...
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng,../input/shopee-product-matching/test_images/0...,1011100101001100101100000000111011010011111001...


# Label Group

In [11]:
label_group_count = train_df.groupby(['label_group']).size().reset_index()
label_group_count.columns = ['label_group', 'count']
label_group_count.sort_values(by='count', ascending=False, inplace=True)
label_group_count

,label_group,count
3041,1163569239,51
412,159351600,51
2606,994676122,51
8037,3113678103,51
9370,3627744656,51
...,...,...
6069,2357508171,2
6068,2357372960,2
6067,2357221297,2
6066,2355878351,2


In [12]:
print(f'No. of Duplicate label group: {train_df[train_df["label_group"].duplicated() == True].shape[0]}')
print(f"Minimum product under single label group: {label_group_count['count'].min()}\nMaximum product under single label group: {label_group_count['count'].max()}")

No. of Duplicate label group: 23236
Minimum product under single label group: 2
Maximum product under single label group: 51


In [13]:
x, y = label_group_count['label_group'][:50], label_group_count['count'][:50]
plot_bar_chart(x, y, title='Label Group Chart')

NameError: name 'sns' is not defined

<Figure size 1440x1080 with 0 Axes>